In [1]:
import numpy as np
import matplotlib.pyplot as plt
from scipiy import stats
from datetime import datetime, timedelta

# Load functions to load data and make decision trees
%run ./load_nc_and_subset.ipynb
%run ./DecisionTree.ipynb

In [2]:
# Function to create and train a random forest

# Function to make predictions using a random forest


In [3]:
# Function to create and train a boosted ensemble model

# Function to make predictions using a boosted ensemble model


In [5]:
# Load the data
sesrFName = 'sesr_all_years_USDMTimeScale_conus.nc'
spiFName  = 'SPI_all_years_USDMTimeScale_conus.nc'
usdmFName = 'USDM_grid_all_years.nc'

sesrSName = 'sesr'
spiSName  = 'SPI'
usdmSName = 'USDM'

sesr = LoadNC(sesrFName, sesrSName)
spi  = LoadNC(spiFName, spiSName)
usdm = LoadNCnomask(usdmFName, usdmSName)

In [20]:
# Prepare the data

# Collect the training data (2019, a null year, and 2011 an extreme drought year)
TestInd = np.where( (usdm['year'] == 2011) | (usdm['year'] == 2019) )[0]
TrainValInd = np.where( (usdm['year'] != 2011) & (usdm['year'] != 2019) )[0]

sesr_test = sesr['sesr'][:,:,TestInd]
spi_test  = spi['SPI'][:,:,TestInd]
usdm_test = usdm['USDM'][:,:,TestInd]

sesr_TrainVal = sesr['sesr'][:,:,TrainValInd]
spi_TrainVal  = spi['SPI'][:,:,TrainValInd]
usdm_TrainVal = usdm['USDM'][:,:,TrainValInd]

# Collect the training and validation datasets. Use 2017, drought in northern plains, as a validation set
ValInd = np.where(usdm['year'] == 2017)[0]
TrainInd = np.where( (usdm['year'] != 2011) & (usdm['year'] != 2017) & (usdm['year'] != 2019) )[0]

sesr_train = sesr['sesr'][:,:,TrainInd]
spi_train  = spi['SPI'][:,:,TrainInd]
usdm_train = usdm['USDM'][:,:,TrainInd]

sesr_val = sesr['sesr'][:,:,ValInd]
spi_val  = spi['SPI'][:,:,ValInd]
usdm_val = usdm['USDM'][:,:,ValInd]

# Transform the data into 1D arrays for easier iterations in the SL learning.
I, J, T_train = usdm_train.shape
T_val         = usdm_val.shape[-1]
T_train_val   = usdm_TrainVal.shape[-1]
T_test        = usdm_test.shape[-1]

# USDM is the label, therefore is the y vector in the learning algorithms.
y_train     = usdm_train.reshape(I*J*T_train, order = 'F')
y_val       = usdm_val.reshape(I*J*T_val, order = 'F')
y_train_val = usdm_TrainVal.reshape(I*J*T_train_val, order = 'F')
y_test      = usdm_test.reshape(I*J*T_test, order = 'F')


sesr_train1D     = sesr_train.reshape(I*J*T_train, order = 'F')
sesr_val1D       = sesr_val.reshape(I*J*T_val, order = 'F')
sesr_train_val1D = sesr_TrainVal.reshape(I*J*T_train_val, order = 'F')
sesr_test1D      = sesr_test.reshape(I*J*T_test, order = 'F')

spi_train1D     = spi_train.reshape(I*J*T_train, order = 'F')
spi_val1D       = spi_val.reshape(I*J*T_val, order = 'F')
spi_train_val1D = spi_TrainVal.reshape(I*J*T_train_val, order = 'F')
spi_test1D      = spi_test.reshape(I*J*T_test, order = 'F')


# Finally, the features are the compination of SESR and SPI
# For x, the first column is SESR, the second is SPI. shape[-1] is the number of features
x_train     = np.asarray([sesr_train1D, spi_train1D]).T
x_val       = np.asarray([sesr_val1D, spi_val1D]).T
x_train_val = np.asarray([sesr_train_val1D, spi_train_val1D]).T
x_test      = np.asarray([sesr_test1D, spi_test1D]).T

(8581860,) (8581860,)
(7511700, 2)
